In [1]:
import numpy as np
from datetime import datetime
import ray

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import time

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2018-12-18_16-25-34_80228/logs.


Waiting for redis server at 127.0.0.1:10290 to respond...


Waiting for redis server at 127.0.0.1:14358 to respond...


Starting the Plasma object store with 6.871947672999999 GB memory using /tmp.


{'node_ip_address': '192.168.101.161',
 'redis_address': '192.168.101.161:10290',
 'object_store_addresses': ['/tmp/ray/session_2018-12-18_16-25-34_80228/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2018-12-18_16-25-34_80228/sockets/raylet'],
 'webui_url': ''}

In [2]:

ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2018-12-18_16-25-34_80228/logs.


Waiting for redis server at 127.0.0.1:10290 to respond...


Waiting for redis server at 127.0.0.1:14358 to respond...


Starting the Plasma object store with 6.871947672999999 GB memory using /tmp.


{'node_ip_address': '192.168.101.161',
 'redis_address': '192.168.101.161:10290',
 'object_store_addresses': ['/tmp/ray/session_2018-12-18_16-25-34_80228/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2018-12-18_16-25-34_80228/sockets/raylet'],
 'webui_url': ''}

In [34]:
@ray.remote
class RayProc(object):
    def __init__(self):
        self._images = []
        
    def set_image(self,image: np.array):
        self._images.append(image)
    
    def get_image(self, idx):
        return self._images[idx]
    
    def simple_proc(self):
        self._images[-1] = np.sqrt(self._images[-1])


In [43]:

processor= RayProc.remote()

In [44]:
@ray.remote
def run_expt(idx, img, process):
    process.set_image.remote(img)
    if idx%2==0:
        process.simple_proc.remote()

In [45]:
images = [16*np.ones(shape=(512,512)) for k in range(0,5)]

In [46]:
test = [run_expt.remote(i, images[i], processor) for i in range(0,5)]


In [47]:
ray.get(processor.get_image.remote(0))
ray.get(processor.get_image.remote(1))
ray.get(processor.get_image.remote(2))

array([[4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       ...,
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.]])

array([[16., 16., 16., ..., 16., 16., 16.],
       [16., 16., 16., ..., 16., 16., 16.],
       [16., 16., 16., ..., 16., 16., 16.],
       ...,
       [16., 16., 16., ..., 16., 16., 16.],
       [16., 16., 16., ..., 16., 16., 16.],
       [16., 16., 16., ..., 16., 16., 16.]])

array([[4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       ...,
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.]])